# Обучение модели Stable Diffusion с помощью DreamBooth и LoRA

In [ ]:
import os
import numpy as np
import random
import torch

In [ ]:
SEED = 42

In [ ]:
def seed_everything(TORCH_SEED):
	random.seed(TORCH_SEED)
	np.random.seed(TORCH_SEED)
	torch.manual_seed(TORCH_SEED)
	torch.cuda.manual_seed_all(TORCH_SEED)
	torch.backends.cudnn.deterministic = True
	torch.backends.cudnn.benchmark = False

In [ ]:
seed_everything(SEED)

## Training

In [ ]:
PRETRAINED_MODEL = "runwayml/stable-diffusion-v1-5"

INSTANCE_PROMPT = "a sks lemur"
CLASS_PROMPT = "lemur"

NUM_IMAGES = 6

FP_16 = True
BATCH_SIZE = 1

LEARNING_RATE = 0.001
STEPS = 1200
WITH_PRIOR_PRESERVATION = False
PRIOR_LOSS_WEIGHT = 0.001
LORA_RANK = 16

NEW_LEARNING_RATE = LEARNING_RATE / BATCH_SIZE

INSTANCE_DIR = f"./data/images/train/{NUM_IMAGES}"
CLASS_DIR = "./data/class_images"
LOSS_SUBSTRING = f"with_pp-loss_weight_{PRIOR_LOSS_WEIGHT}" if WITH_PRIOR_PRESERVATION else "without_pp"
OUTPUT_DIR = f"./checkpoints/output-images_{NUM_IMAGES}-{LOSS_SUBSTRING}-lr_{NEW_LEARNING_RATE}-numsteps_{STEPS}-rank_{LORA_RANK}"

RESOLUTION = 512

if not os.path.exists(str(OUTPUT_DIR)):
    %mkdir -p "$OUTPUT_DIR"

In [ ]:
if FP_16:
  fp_16_arg = "fp16"
else:
  fp_16_arg = "no"

if WITH_PRIOR_PRESERVATION:
  command = (f'CUDA_DEVICE_ORDER=PCI_BUS_ID CUDA_VISIBLE_DEVICES=5 accelerate launch train_dreambooth_lora.py '
             f'--pretrained_model_name_or_path="{PRETRAINED_MODEL}" '
             f'--instance_data_dir="{INSTANCE_DIR}" '
             f'--class_data_dir="{CLASS_DIR}" '
             f'--output_dir="{OUTPUT_DIR}" '
             f'--instance_prompt="{INSTANCE_PROMPT}" '
             f'--class_prompt="{CLASS_PROMPT}" '
             f'--seed="{SEED}" '
             f'--resolution=512 '
             f'--use_8bit_adam '
             f'--mixed_precision="{fp_16_arg}" '
             f'--train_batch_size=1 '
             f'--gradient_accumulation_steps=1 '
             f'--with_prior_preservation '
             f'--prior_loss_weight={PRIOR_LOSS_WEIGHT} '
             f'--learning_rate={NEW_LEARNING_RATE} '
             f'--lr_scheduler="constant" '
             f'--lr_warmup_steps=0 '
             f'--max_train_steps={STEPS} '
             f'--rank={LORA_RANK} '
            )
else:
  command = (f'CUDA_DEVICE_ORDER=PCI_BUS_ID CUDA_VISIBLE_DEVICES=5 accelerate launch train_dreambooth_lora.py '
             f'--pretrained_model_name_or_path="{PRETRAINED_MODEL}" '
             f'--instance_data_dir="{INSTANCE_DIR}" '
             f'--output_dir="{OUTPUT_DIR}" '
             f'--instance_prompt="{INSTANCE_PROMPT}" '
             f'--seed="{SEED}" '
             f'--resolution=512 '
             f'--use_8bit_adam '
             f'--mixed_precision="{fp_16_arg}" '
             f'--train_batch_size=1 '
             f'--gradient_accumulation_steps=1 '
             f'--learning_rate={NEW_LEARNING_RATE} '
             f'--lr_scheduler="constant" '
             f'--lr_warmup_steps=0 '
             f'--max_train_steps={STEPS} '
             f'--rank={LORA_RANK} '
            )

In [ ]:
!{command}

## Inference

In [ ]:
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler

In [ ]:
pipe = DiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe.to("cuda")
pipe.unet.load_attn_procs(f"./{OUTPUT_DIR}/pytorch_lora_weights.safetensors")

In [ ]:
unique_token, class_token = "sks", "lemur"
prompts = [p.format(unique_token, class_token) for p in prompts]

In [ ]:
!mkdir {OUTPUT_DIR}/images

In [ ]:
generator = torch.Generator(device="cuda").manual_seed(SEED)

In [ ]:
for idx, prompt in enumerate(prompts):
    pipe(prompt, num_inference_steps=50, generator=generator).images[0].save(f'./{OUTPUT_DIR}/images/{idx}.jpg')